# 장고 웹 프로그래밍 실습

실습 환경 : Pycharm, python 3.6 ver\
책 & 예제 : 파이썬 웹프로그래밍 chapter 3(개정판, 한빛미디어)

## 1. 프로젝트 생성
터미널에서 처음부터 프로젝트를 생성하자

```
> django-admin startproject mysite(프로젝트 이름)
```
프로젝트 이름의 디렉토리가 위 아래로 생겨서 이름을 바꿔줘도 상관없다.
```
> move myProject(새로운 이름)
```

## 2. 애플리케이션 생성
프로젝트 루트 디렉토리에서 polls라는 애플리케이션을 만든다.

```
> python manage.py startapp polls(어플리케이션 이름)
```
어플리케이션 디렉토리와 하위 파일들이 자동으로 생성되었음을 알 수 있다.
개발자들은 알아서 그 내요을 채워넣기만 하면 된다.

## 3. 프로젝트 설정 파일 변경
프로젝트에 필요한 설정은 settinfs.py에 있다.\
루트 디렉토리를 포함한 각종 디렉토리의 위치, 로그의 형식, 프로젝트에 포함된 애플리케이션의 이름 등이 지정되어 있다. 다음은 프로젝트 설정 파일을 변경하는 순서이다.

1. ALLOWED_HOSTS 항목을 지정해야 한다. 장고는 DEBUG=True이면 개발 모드이며 False는 운영모드이다. 운영모드인 경우 반드시 서버의 IP나 도메인을 지정해야 하고, 개발 모드인 경우에는 ['localhost','127.0.0.1']로 간주한다.\
개발모드에서 서버 IP가 기존과 다를 수 있다면 아래와 같이 지정한다.

```python
ALLOWED_HOSTS = [ '192.168.56.101', 'localhost', '127.0.0.1' ]
```

2. 프로젝트에 포함되는 애플리케이션은 모두 설정 파일에 등록되어야 한다. 따라서 polls도 등록해야한다. 정확히는 애플리케이션의 설정 클래스로 등록을 하면 된다.\
polls앱의 설정 클래스는 startapp polls 명령 시에 생성된 apps.py에 PollsConfig로 정의되어 있다. 장고가 설정을 찾을 수 있도록 모듈 경로가지 포함하여 등록하자

```python
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'polls.apps.PollsConfig', # 추가되었다.
]
```

3. 프로젝트에 사용할 DB엔진을 설정한다. 장고는 default로 SQLite3를 사용하고 있다. 이 외의 다른 DB를 사용하고 싶다면 settings.py에서 수정하면 된다.
```python
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': os.path.join(BASE_DIR, 'db.sqlite3'),
    }
}
```

4. 타임존 지정을 한다. default는 세계표준시(UTC)이며 한국 시간으로 바꿔준다.
```python
TIME_ZONE = 'Asia/Seoul'
```

## 4. 기본 테이블 생성
기본 테이블을 생성한다. migrate 명령은 DB의 변경사항이 있을 때 이를 반영해주는 명령어이다.
```
>python manage.py migrate
```
아직 테이블을 만들지는 않았으나 장고는 모든 웹 프로젝트 개발 시 반드시 user와 그룹 테이블 등이 필요하다는 가정 하에 설계되었다.\
쉽게 말해서 실질적 데이터가 없더라도 빈 DB파일이 처음부터 필요하다는 뜻 같다.

## 5. 지금까지의 작업 확인하기
현재까지 프로젝트 디렉토리, 애플리케이션 디렉토리를 비롯한 관련 파일들 그리고 user 및 그룹 테이블을 만들었다.\

이를 확인하기 위해서 웹 서버를 실행하고, 접속할 수 있다.

```
>python manage.py runserver
```

터미널에서 나타난 주소(http://127.0.0.1:8000/)에 접속하면 장고 기본 화면이 뜬다.\
이제 테이블을 확인해보자. 로그인을 하기 위하여 계정을 만든다.

```
>python manage.py createsuperuser
```
명령어 실행 이후 나타난 지시에 따라 계정을 생성한다.\
(비밀번호는 일부러 안보여준다 잘 기억하자)\
\
계정을 만들고 난 후 http://127.0.0.1:8000/admin 에 접속하면 테이블을 확인할 수 있다.\
앞으로 이 사이트에서 user와 group 테이블을 포함하여 데이터를 조작할 수 있다.

# Model 코딩

모델 작업은 DB에 테이블을 생성하는 작업이다.\
모델링은 책의 예제대로 따라간다. 진행은 다음 순서와 같이 진행된다.

## 1. 테이블 정의
polls 애플리케이션은 Question과 Choice 두 개의 테이블이 필요하다(예제의 요구사항이다).\
테이블은 models.py에 정의한다. 

In [ ]:
# models.py
from django.db import models 
class Questions(models.Model):
    question_text = models.CharField(max_length=200)
    pub_date = models.DateTimeField('date published')
    
    # 객체를 문자열로 표현할 때 사용한다. Admin사이트나 장고 쉘 등에서 
    # 테이블명을 제대로 표현하기 위해서 필수적이다
    def __str__(self):
        return self.question_text

class Choice(models.Model):
    question = models.ForeignKey(Question, on_delete=models.CASCADE)
    choice_text = models.CharField(max_length=200)
    votes = models.IntegerField(default=0)
    
    def __str__(self):
        return self.choice_text


장고에서는 테이블을 하나의 클래스로 정의하고, column은 클래스의 변수로 매핑한다. 테이블 클래스는 django.db.models.Model을 상속받으며 각 변수의 타입도 미리 정의된 필드 클래스를 사용한다.

* Question 테이블 컬럼과 클래스 변수 간 매핑

| 테이블 컬럼명 | 컬럼 타입 | 장고의 클래스 변수 | 장고의 필드 클래스 |
| :------------ | :-------- | :----------------- | :----------------- |
| id            | integer   | (id)               | (PK는 장고에서 자동 생성해줌) |
| question_text | varchar(200) | question_text   | models.CharField(max_legnth=200) |
| pub_date      | datetime  | pub_date           | models.DateTimeField('date published') |

* Choice 테이블 컬럼과 클래스 변수 간 매핑

| 테이블 컬럼명 | 컬럼 타입 | 장고의 클래스 변수 | 장고의 필드 클래스 |
| :------------ | :-------- | :----------------- | :----------------- |
| id            | integer   | (id)               | (PK는 장고에서 자동 생성해줌) |
| choice_text   | varchar(200) | choice_text    | models.CharField(max_legnth=200) |
| votes         | integer   | votes              | models.IntegerField(default=0) |
| question_id   | integer   | question           | models. ForeignKey(Question) |

## 2. Admin 사이트에 테이블 반영
이제 models.py 에서 정의한 테이블도 등록해보자

In [ ]:
# admin.py
from django.contrib import admin
from polls.models import Question, Choice

admin.site.register(Question)
admin.stie.register(Choice)

여기까지 하고 manage.py를 실행시키면 된다.

## 3. DB 변경사항 반영
앞 서 DB에 변경사항이 생기면 이를 실제로 반영해줘야 한다고 언급했다. 다음 명령으로 반영하자

```
>python manage.py makemigrations
>python manage.py migrate
```
migrations는 테이블의 변경사항을 알려주는 정보이다. 물리적으로는 애플리케이션 디렉토리별로 존재하며 이 파일들을 이용해 migrate 명령으로 DB에 테이블을 만든다.

마지막으로 admin사이트에 접속하면 변경사항이 모두 적용되었음을 확인할 수 있다.

# 애플리케이션 개발하기 - View 및 Template 코딩

추가 예정